In [16]:
import pandas as pd
import os
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import plotly.graph_objects as go
import ruptures as rpt
from itertools import combinations as comb
from statsmodels.stats import power

In [46]:
folder_path = 'data'
def calculate_mean_for_30s(df):
    df['Time (s)'] = pd.to_datetime(df['Time (s)'], unit='s')
    df.set_index('Time (s)', inplace=True)
    resampled_df = df.resample('30s').mean()
    return resampled_df

def calculate_var_for_30s(df):
    resampled_df = df.resample('30s').var()
    return resampled_df

def read_data(filename):
    accelerometer = pd.read_csv(os.path.join(folder_path, filename, 'Accelerometer.csv'),sep=';')
    gyroscope = pd.read_csv(os.path.join(folder_path, filename,'Gyroscope.csv'),sep=';')
    accelerometer.rename(columns={'Acceleration x (m/s^2)':'accelerometer_x','Acceleration y (m/s^2)':'accelerometer_y','Acceleration z (m/s^2)':'accelerometer_z'}, inplace=True)
    gyroscope.rename(columns={'Gyroscope x (rad/s)':'gyroscope_x','Gyroscope y (rad/s)':'gyroscope_y','Gyroscope z (rad/s)':'gyroscope_z'}, inplace=True)
    accelerometer =accelerometer[9000:-9000]
    gyroscope = gyroscope[9000:-9000]
    accelerometer_mean = calculate_mean_for_30s(accelerometer)
    gyroscope_mean = calculate_mean_for_30s(gyroscope)
    accelerometer_var = calculate_var_for_30s(accelerometer)
    gyroscope_var = calculate_var_for_30s(gyroscope)
    return accelerometer, gyroscope , accelerometer_mean, gyroscope_mean, accelerometer_var, gyroscope_var

def do_test(dataframe1, dataframe2):
    t_stat, p_val = stats.ttest_ind(dataframe1, dataframe2)
    return t_stat, p_val

In [47]:
accelerometer_lucas1, gyroscope_lucas1, accelerometer_lucas1_mean, gyroscope_lucas1_mean, accelerometer_lucas1_var, gyroscope_lucas1_var  = read_data('lucas1')
accelerometer_nick1, gyroscope_nick1, accelerometer_nick1_mean, gyroscope_nick1_mean, accelerometer_nick1_var, gyroscope_nick1_var  = read_data('nick1')
accelerometer_till1, gyroscope_till1, accelerometer_till1_mean, gyroscope_till1_mean, accelerometer_till1_var, gyroscope_till1_var  = read_data('till1')
accelerometer_luisa1, gyroscope_luisa1, accelerometer_luisa1_mean, gyroscope_luisa1_mean, accelerometer_luisa1_var, gyroscope_luisa1_var  = read_data('luisa1')

In [19]:
dict_accel_var = {'accelerometer_lucas1_var': accelerometer_lucas1_var, 'accelerometer_nick1_var': accelerometer_nick1_var, 'accelerometer_till1_var': accelerometer_till1_var, 'accelerometer_luisa1_var': accelerometer_luisa1_var}
dict_accel_mean = {'accelerometer_lucas1_mean': accelerometer_lucas1_mean, 'accelerometer_nick1_mean': accelerometer_nick1_mean, 'accelerometer_till1_mean': accelerometer_till1_mean, 'accelerometer_luisa1_mean': accelerometer_luisa1_mean}
dict_gyro_var = {'gyroscope_lucas1_var': gyroscope_lucas1_var, 'gyroscope_nick1_var': gyroscope_nick1_var, 'gyroscope_till1_var': gyroscope_till1_var, 'gyroscope_luisa1_var': gyroscope_luisa1_var}
dict_gyro_mean = {'gyroscope_lucas1_mean': gyroscope_lucas1_mean, 'gyroscope_nick1_mean': gyroscope_nick1_mean, 'gyroscope_till1_mean': gyroscope_till1_mean, 'gyroscope_luisa1_mean': gyroscope_luisa1_mean}
dict_of_dfs = {'accel_mean': dict_accel_mean, 'gyro_mean':dict_gyro_mean,'accel_var':dict_accel_var, 'gyro_var' :dict_gyro_var}

In [66]:
list_columns_accelerometer = ['accelerometer_x', 'accelerometer_y', 'accelerometer_z']
list_columns_gyroscope = ['gyroscope_x', 'gyroscope_y', 'gyroscope_z']

# die art wie ich hier die results speichere suckt, sollte eigentlich nen table als index haben aber we
results = {}

search_string = 'accelerometer'
t = power.TTestPower()

for df_name, df in dict_of_dfs.items():
    combinations = list(comb(df, 2))
    for combo in combinations:
        if search_string in combo[0]:
            for column in list_columns_accelerometer:
                effect_size =  (df[combo[0]][column].mean()-df[combo[1]][column].mean())/pd.concat([df[combo[0]][column], df[combo[1]][column]]).std()
                nr_participants = t.solve_power(effect_size=effect_size, nobs=None, alpha=0.05, power=0.90, alternative='two-sided')
                t_stat, p_val = do_test(df[combo[0]][column], df[combo[1]][column])
                results[(combo[0],combo[1],column)] = [t_stat, p_val, nr_participants]
        else:
            for column in list_columns_gyroscope:
                t_stat, p_val = do_test(df[combo[0]][column], df[combo[1]][column])
                effect_size =  (df[combo[0]][column].mean()-df[combo[1]][column].mean())/pd.concat([df[combo[0]][column], df[combo[1]][column]]).std()
                nr_participants = t.solve_power(effect_size=effect_size, nobs=None, alpha=0.05, power=0.90, alternative='two-sided')
                results[(combo[0],combo[1],column)] = [t_stat, p_val, nr_participants]

results_df = pd.DataFrame.from_dict(results, orient='index', columns=['T_Statistic', 'P_Value', 'samples_required'])
results_df['minutes_required'] = results_df['samples_required']/2

In [67]:
pd.set_option('display.float_format', '{:.4f}'.format)
results_df[:36]

,T_Statistic,P_Value,samples_required,minutes_required
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_x)",47.3376,0.0000,4.7286,2.3643
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_y)",-2.9718,0.0048,16.0592,8.0296
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_z)",10.5200,0.0000,5.5695,2.7848
"(accelerometer_lucas1_mean, accelerometer_till1_mean, accelerometer_x)",45.9698,0.0000,4.8841,2.4420
"(accelerometer_lucas1_mean, accelerometer_till1_mean, accelerometer_y)",-8.3831,0.0000,6.1252,3.0626
"(accelerometer_lucas1_mean, accelerometer_till1_mean, accelerometer_z)",34.2570,0.0000,4.9182,2.4591
"(accelerometer_lucas1_mean, accelerometer_luisa1_mean, accelerometer_x)",22.2285,0.0000,5.0451,2.5225
"(accelerometer_lucas1_mean, accelerometer_luisa1_mean, accelerometer_y)",-11.9592,0.0000,5.4849,2.7424
"(accelerometer_lucas1_mean, accelerometer_luisa1_mean, accelerometer_z)",63.3717,0.0000,4.8882,2.4441
"(accelerometer_nick1_mean, accelerometer_till1_mean, accelerometer_x)",-21.7040,0.0000,5.2062,2.6031


In [22]:
results_df[36:]

,T Statistic,P Value
"(accelerometer_lucas1_var, accelerometer_nick1_var, accelerometer_x)",47.7645,0.0000
"(accelerometer_lucas1_var, accelerometer_nick1_var, accelerometer_y)",6.9340,0.0000
"(accelerometer_lucas1_var, accelerometer_nick1_var, accelerometer_z)",-13.2708,0.0000
"(accelerometer_lucas1_var, accelerometer_till1_var, accelerometer_x)",51.8719,0.0000
"(accelerometer_lucas1_var, accelerometer_till1_var, accelerometer_y)",16.5029,0.0000
"(accelerometer_lucas1_var, accelerometer_till1_var, accelerometer_z)",-8.3623,0.0000
"(accelerometer_lucas1_var, accelerometer_luisa1_var, accelerometer_x)",29.1915,0.0000
"(accelerometer_lucas1_var, accelerometer_luisa1_var, accelerometer_y)",21.0872,0.0000
"(accelerometer_lucas1_var, accelerometer_luisa1_var, accelerometer_z)",12.6887,0.0000
"(accelerometer_nick1_var, accelerometer_till1_var, accelerometer_x)",12.3039,0.0000


In [25]:
t = power.TTestPower()

In [50]:
t.solve_power(effect_size=0.865871, nobs=None, alpha=0.05, power=0.99, alternative='two-sided')

26.5265118535609